In [118]:
import requests
from bs4 import BeautifulSoup

In [119]:
url = "https://www.imdb.com/search/keyword/?mode=detail&page=1&ref_=kw_ref_typ&title_type=movie&sort=moviemeter,asc"
response = requests.get(url)

In [120]:
soup = BeautifulSoup(response.content,'html.parser')
data = soup.findAll('div',attrs = {'class':'lister-item mode-detail'})

In [121]:
data[0].findAll('p', class_='text-muted text-small')

[<p class="text-muted text-small">
 <span class="certificate">R</span>
 <span class="ghost">|</span>
 <span class="runtime">115 min</span>
 <span class="ghost">|</span>
 <span class="genre">
 Comedy, Drama, Romance            </span>
 </p>,
 <p class="text-muted text-small">
     Director:
 <a href="/name/nm0795290/">Michael Showalter</a>
 <span class="ghost">|</span> 
     Stars:
 <a href="/name/nm0004266/">Anne Hathaway</a>, 
 <a href="/name/nm5954280/">Nicholas Galitzine</a>, 
 <a href="/name/nm5840032/">Ella Rubin</a>, 
 <a href="/name/nm1754239/">Annie Mumolo</a>
 </p>,
 <p class="text-muted text-small">
 <span class="text-muted">Votes:</span>
 <span data-value="27121" name="nv">27,121</span>
 </p>]

In [136]:
data[0].find('div', class_='lister-item-content').findAll('p')[1].get_text(strip=True)

'Solène, a 40-year-old single mom, begins an unexpected romance with 24-year-old Hayes Campbell, the lead singer of August Moon, the hottest boy band on the planet.'

In [137]:
data[0].find('span', class_='metascore favorable').get_text(strip=True)

'67'

In [138]:
data[0].find('div', class_='inline-block ratings-imdb-rating')['data-value']

'6.4'

In [140]:
data[0].find('span', class_='certificate').get_text()

'R'

In [141]:
data[0].find('span', class_='runtime').get_text()

'115 min'

In [142]:
data[0].find('span', class_='genre').get_text(strip=True)

'Comedy, Drama, Romance'

In [143]:
crew = data[0].find_all('p', class_='text-muted text-small')[1]
crew

<p class="text-muted text-small">
    Director:
<a href="/name/nm0795290/">Michael Showalter</a>
<span class="ghost">|</span> 
    Stars:
<a href="/name/nm0004266/">Anne Hathaway</a>, 
<a href="/name/nm5954280/">Nicholas Galitzine</a>, 
<a href="/name/nm5840032/">Ella Rubin</a>, 
<a href="/name/nm1754239/">Annie Mumolo</a>
</p>

In [144]:
crew.find('a').get_text(strip=True)

'Michael Showalter'

In [146]:
[a.get_text(strip=True) for a in crew.find_all('a')[1:]]

['Anne Hathaway', 'Nicholas Galitzine', 'Ella Rubin', 'Annie Mumolo']